# Backend-Based Usage Examples for OmniQ

This notebook demonstrates how to use the backend abstraction layer in OmniQ to simplify configuration while maintaining flexibility. Backends act as unified factories for creating storage components, reducing boilerplate code and configuration complexity.

## What are Backends?

Backends in OmniQ are abstraction layers that provide unified interfaces for storage systems. Instead of configuring each component (task queue, result storage, event storage) individually, you create backend instances that act as factories for all related storage components.

### Key Benefits:
- **Simplified Configuration**: One configuration for multiple components
- **Consistency**: All components share the same storage settings
- **Flexibility**: Mix and match different backends for different components
- **Resource Efficiency**: Shared connections and optimizations


In [ ]:
# Import required modules
import asyncio
import datetime as dt
from pathlib import Path

from omniq import OmniQ, AsyncOmniQ
from omniq.backend import SQLiteBackend, FileBackend
from omniq.queue import TaskQueue
from omniq.results import ResultStore
from omniq.events import EventStore

## Define Example Tasks

Let's define some simple tasks to demonstrate the backend functionality:

In [ ]:
def simple_task(name: str, multiplier: int = 1) -> str:
    """A simple task function for demonstration."""
    result = f"Hello {name}!" * multiplier
    print(f"Task executed: {result}")
    return result


async def async_task(name: str, delay: float = 0.1) -> str:
    """An async task function for demonstration."""
    await asyncio.sleep(delay)
    result = f"Async hello {name}!"
    print(f"Async task executed: {result}")
    return result

## 1. Single Backend Usage

The simplest backend-based approach uses one backend for all components. This is ideal when you want consistency across all storage operations and don't need specialized backends for different components.

### When to use:
- Simple applications with consistent storage needs
- Development and testing environments
- When you want to minimize configuration complexity
- Small to medium-scale applications

In [ ]:
# Create a SQLite backend - all components will use this database
sqlite_backend = SQLiteBackend({
    "db_path": "notebook_single_backend.db",
    "create_dirs": True
})

print(f"Created SQLite backend: {sqlite_backend}")
print(f"Database path: {sqlite_backend.config['db_path']}")

In [ ]:
# Create OmniQ instance from the backend
# All components (queue, result storage, event storage) will use SQLite
oq = OmniQ.from_backend(
    backend=sqlite_backend,
    worker_type="thread_pool",
    worker_config={"max_workers": 4}
)

print("Created OmniQ instance with single SQLite backend")
print(f"Task queue type: {type(oq.task_queue).__name__}")
print(f"Result storage type: {type(oq.result_storage).__name__}")
print(f"Event storage type: {type(oq.event_storage).__name__}")

In [ ]:
# Use the OmniQ instance
with oq:
    # Start the worker
    oq.start_worker()
    
    # Enqueue some tasks
    task_id1 = oq.enqueue(
        func=simple_task,
        func_kwargs={"name": "Alice", "multiplier": 2},
        queue_name="default"
    )
    
    task_id2 = oq.enqueue(
        func=simple_task,
        func_kwargs={"name": "Bob", "multiplier": 1},
        queue_name="default",
        run_in=dt.timedelta(seconds=1)
    )
    
    print(f"Enqueued tasks: {task_id1}, {task_id2}")
    
    # Wait for results
    result1 = oq.get_result(task_id1, timeout=dt.timedelta(seconds=10))
    result2 = oq.get_result(task_id2, timeout=dt.timedelta(seconds=10))
    
    print(f"Task 1 result: {result1.result if result1 else 'None'}")
    print(f"Task 2 result: {result2.result if result2 else 'None'}")
    
    # Stop the worker
    oq.stop_worker()

print("Single backend example completed!")

## 2. Mixed Backend Usage

The mixed backend approach allows you to use different backends for different components. This is powerful when you want to optimize each component for its specific requirements.

### Common patterns:
- **Fast storage for queues**: Use Redis or file storage for temporary task queues
- **Persistent storage for results**: Use SQLite or PostgreSQL for long-term result storage
- **Audit storage for events**: Use separate database for event logging and compliance

### When to use:
- Production environments with specific performance requirements
- When different components have different scalability needs
- Compliance requirements (separate audit trails)
- Existing infrastructure integration

In [ ]:
# Create different backends for different purposes

# Fast file-based backend for task queue (temporary storage)
file_backend = FileBackend({
    "base_dir": "notebook_queue_storage",
    "create_dirs": True
})

# Persistent SQLite backend for result storage (long-term storage)
result_backend = SQLiteBackend({
    "db_path": "notebook_results.db",
    "create_dirs": True
})

# Another SQLite backend for event storage (audit trail)
event_backend = SQLiteBackend({
    "db_path": "notebook_events.db",
    "create_dirs": True
})

print(f"File backend for queue: {file_backend}")
print(f"SQLite backend for results: {result_backend}")
print(f"SQLite backend for events: {event_backend}")

In [ ]:
# Create OmniQ with mixed backends
mixed_oq = OmniQ.from_backend(
    backend=file_backend,              # Task queue uses file storage
    result_store_backend=result_backend,  # Results use SQLite
    event_store_backend=event_backend,    # Events use separate SQLite DB
    worker_type="async",
    worker_config={"max_workers": 6}
)

print("Created OmniQ instance with mixed backends")
print(f"Task queue backend: File storage")
print(f"Result storage backend: SQLite (results.db)")
print(f"Event storage backend: SQLite (events.db)")

In [ ]:
# Use the mixed backend OmniQ instance
with mixed_oq:
    # Start the worker
    mixed_oq.start_worker()
    
    # Enqueue tasks to different queues
    high_priority_task = mixed_oq.enqueue(
        func=simple_task,
        func_kwargs={"name": "Priority User", "multiplier": 3},
        queue_name="high"
    )
    
    normal_task = mixed_oq.enqueue(
        func=simple_task,
        func_kwargs={"name": "Normal User", "multiplier": 1},
        queue_name="default"
    )
    
    print(f"Enqueued high priority task: {high_priority_task}")
    print(f"Enqueued normal task: {normal_task}")
    
    # Wait for results
    high_result = mixed_oq.get_result(high_priority_task, timeout=dt.timedelta(seconds=10))
    normal_result = mixed_oq.get_result(normal_task, timeout=dt.timedelta(seconds=10))
    
    print(f"High priority result: {high_result.result if high_result else 'None'}")
    print(f"Normal result: {normal_result.result if normal_result else 'None'}")
    
    # Stop the worker
    mixed_oq.stop_worker()

print("Mixed backend example completed!")

## 3. Individual Component Creation

Sometimes you need direct access to individual components while still benefiting from backend configuration simplification. This approach gives you the flexibility of component-based usage with the convenience of backend configuration.

### When to use:
- When you need direct component APIs
- Building custom orchestration logic
- Integrating with existing systems
- Advanced use cases requiring component-level control

In [ ]:
# Create a backend for component creation
component_backend = SQLiteBackend({
    "db_path": "notebook_components.db",
    "create_dirs": True
})

# Create individual components from the backend
task_queue = TaskQueue.from_backend(
    backend=component_backend,
    queues=["high", "medium", "low"]
)

result_store = ResultStore.from_backend(component_backend)
event_store = EventStore.from_backend(component_backend)

print(f"Created task queue: {task_queue}")
print(f"Created result store: {result_store}")
print(f"Created event store: {event_store}")

In [ ]:
# Use components directly
with task_queue, result_store, event_store:
    # Enqueue a task directly to the queue
    task_id = task_queue.enqueue(
        func=simple_task,
        func_kwargs={"name": "Component User", "multiplier": 2},
        queue_name="medium"
    )
    
    print(f"Enqueued task directly to queue: {task_id}")
    
    # Check queue size
    queue_size = task_queue.get_queue_size("medium")
    print(f"Queue 'medium' size: {queue_size}")
    
    # List all queues
    queues = task_queue.list_queues()
    print(f"Available queues: {queues}")
    
    # Note: In a real application, you would also create and start a worker
    # to process the enqueued tasks and store results in the result_store

print("Component creation example completed!")

## 4. Async Backend Usage

Backends work seamlessly with both synchronous and asynchronous OmniQ interfaces. Here's how to use backends with AsyncOmniQ for high-performance asynchronous task processing.

### Benefits of Async + Backends:
- **High Performance**: Async I/O operations with optimized backend connections
- **Scalability**: Handle many concurrent tasks efficiently
- **Resource Efficiency**: Non-blocking operations with shared backend resources
- **Flexibility**: Mix async and sync tasks in the same system

In [ ]:
# Async backend example
async def async_backend_example():
    """Demonstrate async backend usage."""
    
    # Create backend
    async_backend = SQLiteBackend({
        "db_path": "notebook_async_backend.db",
        "create_dirs": True
    })
    
    # Create AsyncOmniQ instance from backend
    async_oq = await AsyncOmniQ.from_backend(
        backend=async_backend,
        worker_type="async",
        worker_config={"max_workers": 8}
    )
    
    print(f"Created AsyncOmniQ with backend: {async_backend}")
    
    # Use the AsyncOmniQ instance
    async with async_oq:
        # Start the worker
        await async_oq.start_worker()
        
        # Enqueue async tasks
        async_task_id = await async_oq.enqueue(
            func=async_task,
            func_kwargs={"name": "Async User", "delay": 0.5},
            queue_name="default"
        )
        
        sync_task_id = await async_oq.enqueue(
            func=simple_task,
            func_kwargs={"name": "Mixed User", "multiplier": 2},
            queue_name="default"
        )
        
        print(f"Enqueued async task: {async_task_id}")
        print(f"Enqueued sync task: {sync_task_id}")
        
        # Wait for results
        async_result = await async_oq.get_result(
            async_task_id, 
            timeout=dt.timedelta(seconds=10)
        )
        sync_result = await async_oq.get_result(
            sync_task_id, 
            timeout=dt.timedelta(seconds=10)
        )
        
        print(f"Async task result: {async_result.result if async_result else 'None'}")
        print(f"Sync task result: {sync_result.result if sync_result else 'None'}")
        
        # Stop the worker
        await async_oq.stop_worker()
    
    print("Async backend example completed!")

# Run the async example
await async_backend_example()

## 5. Backend Comparison and Best Practices

Let's compare different approaches and discuss when to use each one:

In [ ]:
# Comparison table
import pandas as pd

comparison_data = {
    'Approach': ['Basic Usage', 'Component-Based', 'Single Backend', 'Mixed Backend'],
    'Configuration Complexity': ['Low', 'High', 'Low', 'Medium'],
    'Flexibility': ['Low', 'Very High', 'Medium', 'High'],
    'Best For': [
        'Simple apps, getting started',
        'Complex systems, maximum control',
        'Consistent storage needs',
        'Optimized production systems'
    ],
    'Resource Efficiency': ['Medium', 'Low', 'High', 'Medium'],
    'Maintenance': ['Easy', 'Complex', 'Easy', 'Medium']
}

df = pd.DataFrame(comparison_data)
print("OmniQ Usage Approach Comparison:")
print(df.to_string(index=False))

## Best Practices for Backend Usage

### 1. Choose the Right Backend for Your Needs
- **SQLite**: Great for development, testing, and small to medium applications
- **PostgreSQL**: Best for production systems requiring ACID compliance and scalability
- **Redis**: Excellent for high-performance caching and temporary storage
- **File**: Good for simple deployments and cloud storage integration
- **NATS**: Ideal for distributed systems and microservices

### 2. Single vs Mixed Backend Decision Matrix
- **Use Single Backend when**:
  - All components have similar performance requirements
  - You want to minimize operational complexity
  - Resource sharing and consistency are priorities
  - You're in development or testing phases

- **Use Mixed Backends when**:
  - Different components have different performance needs
  - You need to integrate with existing infrastructure
  - Compliance requires separate audit trails
  - You want to optimize each component independently

### 3. Configuration Management
- Use environment variables for sensitive configuration (passwords, keys)
- Keep backend configurations in version control
- Use different configurations for different environments (dev, staging, prod)
- Document your backend choices and rationale

### 4. Monitoring and Maintenance
- Monitor backend performance and resource usage
- Set up alerts for backend failures or performance degradation
- Plan for backup and recovery of persistent backends
- Regular maintenance and optimization of database backends

## Cleanup

Let's clean up the example files created during this demonstration:

In [ ]:
# Clean up example database files
import shutil

example_files = [
    "notebook_single_backend.db",
    "notebook_results.db",
    "notebook_events.db",
    "notebook_components.db",
    "notebook_async_backend.db"
]

for file_path in example_files:
    try:
        Path(file_path).unlink(missing_ok=True)
        print(f"Cleaned up: {file_path}")
    except Exception as e:
        print(f"Could not clean up {file_path}: {e}")

# Clean up queue storage directory
try:
    shutil.rmtree("notebook_queue_storage", ignore_errors=True)
    print("Cleaned up: notebook_queue_storage directory")
except Exception as e:
    print(f"Could not clean up notebook_queue_storage: {e}")

print("\nCleanup completed!")

## Summary

This notebook demonstrated the backend-based approach to using OmniQ, which provides:

1. **Simplified Configuration**: Backends reduce boilerplate code and configuration complexity
2. **Flexibility**: Choose between single backend simplicity or mixed backend optimization
3. **Resource Efficiency**: Shared connections and backend-specific optimizations
4. **Scalability**: Easy to scale and modify backend configurations as needs change

The backend-based approach strikes an excellent balance between the simplicity of basic usage and the flexibility of component-based usage, making it ideal for most production applications.

### Next Steps
- Explore other OmniQ examples in the `examples/` directory
- Read the OmniQ documentation for advanced configuration options
- Try different backend combinations for your specific use case
- Consider performance testing with your expected workload